In [1]:
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt
import pmdarima as pm
from pmdarima import model_selection
from sklearn.metrics import mean_squared_error
from pmdarima.arima import ADFTest
from pmdarima.arima import AutoARIMA
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pywt
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import mean_squared_error
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
from itertools import chain

D:\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('datacount7.csv',parse_dates = ['ds'])
print(type(data))
print(data.dtypes)
data = data[(data['ds'] < '2022-12-10') | (data['ds'] > '2023-01-20')]

<class 'pandas.core.frame.DataFrame'>
ds    datetime64[ns]
y              int64
dtype: object


In [3]:
data = data['y'].tolist()
actual = data[-31:]

In [4]:
for family in pywt.families():
    print("%s family: " % family + ', '.join(pywt.wavelist(family)))

haar family: haar
db family: db1, db2, db3, db4, db5, db6, db7, db8, db9, db10, db11, db12, db13, db14, db15, db16, db17, db18, db19, db20, db21, db22, db23, db24, db25, db26, db27, db28, db29, db30, db31, db32, db33, db34, db35, db36, db37, db38
sym family: sym2, sym3, sym4, sym5, sym6, sym7, sym8, sym9, sym10, sym11, sym12, sym13, sym14, sym15, sym16, sym17, sym18, sym19, sym20
coif family: coif1, coif2, coif3, coif4, coif5, coif6, coif7, coif8, coif9, coif10, coif11, coif12, coif13, coif14, coif15, coif16, coif17
bior family: bior1.1, bior1.3, bior1.5, bior2.2, bior2.4, bior2.6, bior2.8, bior3.1, bior3.3, bior3.5, bior3.7, bior3.9, bior4.4, bior5.5, bior6.8
rbio family: rbio1.1, rbio1.3, rbio1.5, rbio2.2, rbio2.4, rbio2.6, rbio2.8, rbio3.1, rbio3.3, rbio3.5, rbio3.7, rbio3.9, rbio4.4, rbio5.5, rbio6.8
dmey family: dmey
gaus family: gaus1, gaus2, gaus3, gaus4, gaus5, gaus6, gaus7, gaus8
mexh family: mexh
morl family: morl
cgau family: cgau1, cgau2, cgau3, cgau4, cgau5, cgau6, cgau7, 

In [5]:
# 通过循环设置不同的阈值值，并记录结果
threshold_values = range(1, 401,20)  # 阈值范围从1到400

results = []  # 用于存储每个阈值值对应的结果

In [ ]:
for db_num in range(1, 39):
    for levels in range(1, 7):
        wavelet = f'db{db_num}'
        coeffs = pywt.wavedec(data, wavelet, level=levels)
        for threshold in threshold_values:
            coeffs = [pywt.threshold(c, threshold, 'soft') for c in coeffs]

            meta = pywt.waverec(coeffs, wavelet)
            train = meta[:-31]

            model = auto_arima(train, d=1, D=0, start_p=0, start_q=0, max_p=12, max_q=12,
                               start_P=0, start_Q=0, max_P=12, max_Q=12, seasonal=True, m=7,
                               information_criterion='aic')

            forecast = model.predict(n_periods=31)
            # 假设有实际的值 actual

            mae = mean_absolute_error(actual, forecast)
            rmse = np.sqrt(mean_squared_error(actual, forecast))
            r_squared = r2_score(actual, forecast)
            print(db_num,levels,threshold, mae, rmse, r_squared)
            results.append((db_num,levels,threshold, mae, rmse, r_squared))

1 1 1 165.05745346285246 203.92404309349413 -0.32555541011128586
1 1 21 182.76553505160658 208.55723015410436 -0.3864733336528494
1 1 41 201.34271206996735 219.8092867972141 -0.5401148013667061


In [ ]:
results_df = pd.DataFrame(results, columns=['Threshold', 'MAE', 'RMSE', 'R_squared'])
print(results_df)